In [1]:
## Import libraries

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
import spacy
from matplotlib.ticker import FuncFormatter
from concurrent.futures import ThreadPoolExecutor

## Formatting

# Plots
plt.rc("text", usetex = True) # Use LaTeX to render text
plt.rc("font", family = "serif", size = 12) # Set font family and size

ModuleNotFoundError: No module named 'spacy'

### Save and load data frames

In [14]:
# Save data frames to not re-load them in a new session
# df.to_pickle("preprocessed_df.pkl")
# df_sentiment_index.to_pickle("sentiment_index.pkl")

In [4]:
# Load data frames from previous sessions
df = pd.read_pickle("preprocessed_df.pkl")
df_sentiment_index = pd.read_pickle("sentiment_index.pkl")

### Import earnings call transcripts, sentiment dictionary, dictionary of country names, and financial data 

In [ ]:
## Import earnings conference call transcripts 

# Define function to extract quarter and year information from transcripts
def extract_quarter_year(transcript):
    
    # Regular expressions for quarter, half-year, and full-year
    quarter_pattern = r"Q\d\s*"
    half_year_pattern = r"(?:Half|H1|H2)\s*Year\s*"
    full_year_pattern = r"Full\s*Year\s*"
    year_pattern = r"\d{4}"

    # Search for all patterns and store their match objects and starting positions
    matches = [(re.search(pattern, transcript), pattern) for pattern in (quarter_pattern, half_year_pattern, full_year_pattern)]
    matches = [(match, pattern) for match, pattern in matches if match]

    # Get the first occurring match
    first_match = min(matches, key=lambda x: x[0].start()) if matches else (None, None)

    # Extract quarter, half year, and full year information based on the first occurring match
    if first_match[1] == quarter_pattern:
        quarter = first_match[0].group(0).strip()
        # Validate the quarter value
        quarter_number = int(quarter[1:])
        if quarter_number < 1 or quarter_number > 4:
            quarter = None
    elif first_match[1] == half_year_pattern:
        quarter = "HY"
    elif first_match[1] == full_year_pattern:
        quarter = "FY"
    else:
        quarter = None

    # Extract year information and store in year variable
    year_match = re.search(year_pattern, transcript)
    year = year_match.group(0) if year_match else None

    return quarter, year

# Define function to extract date of conference call and company name from file name and text from txt files
def extract_data_from_file(file_path):
    
    # Extract date and company name from file name
    date_list = os.path.basename(file_path).split("-")[:3]
    date_str = "-".join(date_list)
    company = os.path.basename(file_path).split("-")[3]
    
    # Transform date string into datetime object and remove time
    date = pd.to_datetime(date_str, format = "%Y-%b-%d").date()

    # Extract text from txt files
    with open(file_path, "r", encoding = "utf-8") as f:
        transcript = f.read()
        quarter, year = extract_quarter_year(transcript)

    return {"date": date, "quarter": quarter, "year": year, "company": company, "transcript": transcript}

# Define function to get the date of the quarter end of the quarter discussed in the earnings conference call
def get_quarter_end(row):
    
    # For full year, set quarter end to end of the discussed year
    if row["quarter"] == "FY":
        return pd.Timestamp(year=int(row["year"]), month=12, day=31)
    
    # For half year, set quarter end to the end of that half year of the date of the earnings conference call
    elif row["quarter"] == "HY":
        if row["date"].month <= 6:
            return pd.Timestamp(year=row["date"].year, month=6, day=30)
        else:
            return pd.Timestamp(year=row["date"].year, month=12, day=31)
    
    # For quarterly calls, set date to end of respective quarter
    else:
        return pd.to_datetime(row["quarter_end"]) + pd.offsets.QuarterEnd()

# Define directory
directory = r"C:\Users\fabia\Dropbox\Dokumente\4_UoC CGS-E\2_Dissertation\Projects\5_Bond yield spreads\Data\Earnings conference call transcripts\Eikon 2002 - 2022"

# Initialize list to store data
data_list = []

# Loop over files in directory and subdirectories and apply function to extract data from files
for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith(".txt"):
            file_path = os.path.join(root, file)
            data_list.append(extract_data_from_file(file_path))

# Create data frame from the list
df = pd.DataFrame(data_list).sort_values(by="date").reset_index(drop=True)

# Combine "year"and "quarter" columns into a new "quarter_end" column
df["quarter_end"] = df["year"].astype(str) + df["quarter"]

# Apply function and set quarter end date as index, and drop quarter_end column
df.index = df.apply(get_quarter_end, axis=1)
df.index.name = "quarter_end"
df = df.drop("quarter_end", axis = 1)

# Filter out entries for which there is no "quarter" value
df = df[df['quarter'].notnull()]

## Filter out full year reports, when there are half year reports for the same company in the same year

# Select half year calls
hy_calls = df[df["quarter"] == "HY"]

# Create a set of tuples containing the company and the year of half year calls
hy_company_year = set(hy_calls[["company", "year"]].apply(tuple, axis=1))

# Define function that returns rows that have both full year and half year calls for the same company and year
def to_remove(row):
    return (row["quarter"] == "FY") and ((row["company"], row["year"]) in hy_company_year)

# Apply the function to each row and get a boolean series indicating whether to remove the row or not
fy_rows_to_remove = df.apply(to_remove, axis=1)

# Remove full year entries for which there are also half year entries 
df = df[~fy_rows_to_remove]

# Add column alternative to quarter_end that takes the quarter end of the quarter previous to the date of the conference instead of from transcript - might deal better with companies having different financial year ends
df["date_quarter"] = df["date"] - pd.offsets.QuarterEnd(n=1)
df.insert(1, "date_quarter", df.pop("date_quarter"))

In [25]:
## Import dictionary of country names

# Load country file and convert to string
country_names = pd.read_excel(r"data\country_names\country_names.xlsx")
country_names = country_names.astype(str)

# Create list with country names
countries_list = country_names.iloc[:, 0].tolist()

In [12]:
## Import Loughran & McDonald sentiment dictionary

# Import word and positive/negative tone value from dictionary
sentiment_dict = pd.read_csv(r"data\sentiment_dictionary\Loughran-McDonald_MasterDictionary_1993-2021.csv", usecols = ["Word", "Negative", "Positive"])

# Convert words to lowercase
sentiment_dict["Word"] = sentiment_dict["Word"].str.lower()

# Create python dictionary for faster lookups later
sentiment_dict = {row["Word"]: row["Positive"] - row["Negative"] for _, row in sentiment_dict.iterrows()}

# Transform negative, positive, and neutral sentiment scores into -1, +1, and 0, as current values represent year added
sentiment_dict = {key: (1 if value > 0 else -1 if value < 0 else 0) for key, value in sentiment_dict.items()}

In [26]:
## Import financial data

# Define funtion to read quarterly data
def read_quarterly(filename):
    df = pd.read_csv(os.path.join(r"data\financial_data", filename))
    
    # Transform to datetime
    df["Date"] = pd.to_datetime(df["Date"]) + pd.offsets.QuarterEnd() 
    
    # Set date as index
    df.set_index("Date", inplace=True)
    
    # Sort by ascending date
    df = df.sort_values(by = "Date")
    
    return df

# Define function to read monthly data and transform into quarterly
def read_monthly(filename):
    df = pd.read_csv(os.path.join(r"data\financial_data", filename))

    # Transform to datetime
    df["Date"] = pd.to_datetime(df["Date"], format="%Y%b")
    
    # Set date as index
    df.set_index("Date", inplace = True)
    
    # Average monthly values to construct quarterly values
    df = df.resample("Q").mean(numeric_only=True)
    
    return df

# GDP
gdp = read_quarterly("gdp_EU.csv")

# Year-on-year change in gdp
gdp_yoy = gdp.pct_change(periods = 4)

# Debt-to-gdp
debt_to_gdp = read_quarterly("debt-to-gdp_EU.csv")

# Year-on-year change in debt-to-gdp
debt_to_gdp_yoy = debt_to_gdp.pct_change(periods = 4)

# Current account
current_account = read_quarterly("current account_EU.csv")

# Harmonized index of consumer prices
hicp = read_monthly("hicp_EU_monthly.csv")

# Officical reserves
official_reserves = read_monthly("official reserves_EU_monthly.csv")

# Construct current account / gdp
current_account_over_gdp = current_account.div(gdp)

# Construct official reserves / gdp
official_reserves_over_gdp = official_reserves.div(gdp)

# EURO STOXX 50 & VSTOXX
stoxx = pd.read_excel(r"data\financial_data\stoxx50_vstoxx.xlsx")
stoxx["Date"] = pd.to_datetime(stoxx["Date"]) + pd.offsets.QuarterEnd()
stoxx.set_index("Date", inplace = True)

# Bond yields - missing estonia, latvia, and luxembourg
yields = pd.read_excel(r"data\financial_data\eu_yields_10y.xlsx")
yields["date"] = pd.to_datetime(yields["date"]) + pd.offsets.QuarterEnd()
yields.set_index("date", inplace = True)

# Bond yield spreads - subtracting German bond yield
yield_spreads = pd.DataFrame()
for col in yields.columns:
    if col == "germany":
        continue
    yield_spread = yields[col].sub(yields["germany"])
    yield_spreads[col] = yield_spread

In [27]:
## Import country long-term credit ratings

# Initialize an empty data frame to store the data
ratings = pd.DataFrame()

# Loop over each file in the directory
for filename in os.listdir(r"data\financial_data\ratings"):
    if filename.endswith(".xlsx") and filename.startswith("Ratings_"):
        # Read the excel file into a pandas data frame
        filepath = os.path.join(r"data\financial_data\ratings", filename)
        temp_ratings = pd.read_excel(filepath, usecols=["Date", "Issuer Rating", "Rating Source"])
        
        # Filter out rows with NULL values in the 'Date' column or 'WD' / "RD" in the 'Issuer Rating' column
        temp_ratings = temp_ratings[pd.notnull(temp_ratings["Date"]) & (~temp_ratings["Issuer Rating"].isin(["WD", "RD", "NR"]))]
        
        # Convert the 'Date' column to datetime format and set as index
        temp_ratings["Date"] = pd.to_datetime(temp_ratings["Date"], format = "%d.%m.%Y")
        temp_ratings.set_index('Date', inplace=True)
        
        # Extract the word after "Ratings_" in the file name and rename the columns
        country = filename.split("_", 1)[1].split(".")[0]
        temp_ratings.rename(columns={"Issuer Rating": "rating", "Rating Source": "source"}, inplace=True)
        temp_ratings["country"] = country
        
        # Concatenate the temporary data frame to the main data frame
        ratings = pd.concat([ratings, temp_ratings], ignore_index = False)

# Sort the index of the final data frame in ascending order
ratings.sort_index(inplace=True)

In [28]:
## Convert ratings to numerical scale, combine Moody's and Fitch ratings and aggregate to quarterly data

# Import conversion scale
conversion_ratings = pd.read_excel(r"data\financial_data\ratings\conversion_ratings.xlsx")

# Convert ratings to numerical scale
merged_df = ratings.merge(conversion_ratings, on = ["source", "rating"], how = "left")
merged_df = merged_df.drop(columns = ["rating"]).rename(columns = {"score": "rating"})
merged_df.index = ratings.index
ratings.update(merged_df["rating"])

# Transform into quarterly data
ratings.index = ratings.index.to_series().apply(lambda x: x + pd.offsets.QuarterEnd())

# Combine Moody's and Fitch ratings
ratings = ratings.reset_index()
ratings = ratings.groupby(["Date", "country"])["rating"].mean().reset_index()
ratings = ratings.set_index("Date")

# Make format structurally similar to other data
ratings = pd.pivot_table(ratings, values = "rating", index = ratings.index, columns = "country")

# Fill in rating for quarters, where rating has not been changed
ratings = ratings.fillna(method = "ffill", axis = 0)

### Transcript preprocessing

In [ ]:
# Load the English model of spacy
nlp = spacy.load("en_core_web_sm")

# Define function for text preprocessing of earnings conference call transcripts
def preprocess(text):
    
    # Remove content before "Presentation"
    text = re.sub(r".*Presentation\n-*\n", "", text, flags = re.DOTALL)
    
    # Remove content after "PRELIMINARY TRANSCRIPT:"
    text = re.sub(r"PRELIMINARY TRANSCRIPT:.*", "", text, flags=re.DOTALL)
    
    # Remove last word
    text = re.sub(r'\b\w+\b(?=[^\w]*$)', '', text)
    
    # Remove lines with description of speakers in Q&A (identification based on number in square brackets)
    text = re.sub(r"^.*\[\d+\].*$\n?", "", text, flags = re.MULTILINE)
    
    # Remove all "-" and "=" that appear more than two times after another
    text = re.sub(r"[=-]{3,}", "", text)
    
    # Remove dates
    text = re.sub(r"\d{1,2}(\s*(th|st|nd|rd))?(\s*(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec))?(\s*\d{2,4})?", "", text)
    
    # Remove numbers
    text = re.sub(r"\d+", "", text)
    
    # Convert to lowercase and remove non-alphabetic characters
    text = re.sub(r"[^a-zA-Z\s]", "", text.lower())

    # Tokenize text and remove stop words using spaCy
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_stop and token.is_alpha]
    
    return tokens

In [ ]:
# Define function to run preprocessing on multiple CPU cores to improve speed
def apply_preprocess_parallel(df, func, column_name, num_threads=4):
    
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        
        # Apply preprocessing function in parallel using 4 threads
        results = list(executor.map(func, df[column_name]))
    
    return results

In [ ]:
## Preprocessing of earnings conference call transcripts

# Only run function if it has not been run before
if "transcript_preprocessed" not in df.columns:
    
    # Apply preprocessing function to all transcripts using 4 threads
    df["transcript_preprocessed"] = apply_preprocess_parallel(df, preprocess, "transcript", num_threads=4)
    
    # Delete unprocessed transcripts
    df.drop("transcript", axis=1, inplace=True)

### Sentiment analysis of transcripts

In [27]:
# Define function to get sentiment score of a word from sentiment dictionary
def get_sentiment_score(word):
    
    # Retrieve sentiment score of a word; if word is not in the dicitonary, return 0
    return sentiment_dict.get(word, 0)

In [28]:
# Define function to get aggregate sentiment score (sentiment words +/- 10 words) associated with country in transcript
def get_country_sentiment(transcript, country_word):
    
    # Initialize empty list to store sentiment score
    sentiment_scores = []
    
    # Loop over each token in transcript using "enumerate" to keep track of position of token in transcript
    for i, token in enumerate(transcript):
        
        # Proceed if token matches country word
        if token == country_word:
            
            # Define context window in which sentiment words around country are counted
            start = max(0, i - 10)
            end = min(len(transcript), i + 11)
            
            # Create list that contains all tokens - except country word - in context window
            context = [transcript[j] for j in range(start, end) if j != i]
            
            # Assign sentiment score to country word based on sentiment words in context window
            sentiment_scores.append(sum([get_sentiment_score(token) for token in context]))
    
    # Return sum of sentiment scores
    return sum(sentiment_scores)

In [ ]:
# Define function to count country words per transcript
def count_country_words(transcript, country_names):
    
    # Count 1 if a word in transcript is in the country names dictionary
    return sum(1 for word in transcript if any(word in country_words for country_words in country_names.values))

# Apply function and create column in transcript data frame with total number of country ocurrences in transcript
df["country_count"] = df["transcript_preprocessed"].apply(lambda x: count_country_words(x, country_names))

In [ ]:
# Define function to get total sentiment score for each country per transcript 
def get_total_sentiment_score(transcript):
    
    # Initialize empty dictionary
    total_sentiment_score = {}
    
    # Loop over country names
    for country_words in country_names.values:
        
        # Assign country and its equivalents its total number of ocurrences per transcript
        total_sentiment_score[country_words[0]] = sum(get_country_sentiment(transcript, word) for word in country_words)
    
    return total_sentiment_score

# Calculate total sentiment scores for each country in a separate data frame
sentiment_scores_df = df['transcript_preprocessed'].apply(lambda x: pd.Series(get_total_sentiment_score(x)))

# Concatenate the separate data frame with sentiment scores to the transcripts data frame
df = pd.concat([df, sentiment_scores_df], axis=1)

In [ ]:
# Weighting of scores: divide country sentiment score by total number of discussed countries
for country in countries_list:
    df.loc[df[country] != 0, country] = df.loc[df[country] != 0, country] / df.loc[df[country] != 0, "country_count"]

In [ ]:
## Distribute sentiment scores of half and full year calls over the quarters - MIGHT NEED TO ADAPT TO DATE_QUARTER

# Initialize a list to store the new rows
rows_to_concat = []

# Iterate over the rows of the original DataFrame
for _, row in df.iterrows():
    if row["quarter"] in ["Q1", "Q2", "Q3", "Q4"]:
        rows_to_concat.append(row.to_frame().T)
    elif row["quarter"] == "HY":
        rows_to_concat.append(row.to_frame().T)
        new_row = row.copy()
        new_date = row.name - pd.offsets.QuarterEnd(n=1)
        new_row.name = new_date
        rows_to_concat.append(new_row.to_frame().T)
    elif row["quarter"] == "FY":
        for i in range(4):
            new_row = row.copy()
            new_date = row.name - pd.offsets.QuarterEnd(n=i)
            new_row.name = new_date
            rows_to_concat.append(new_row.to_frame().T)

# Concatenate the new rows and sort the resulting data frame by the index
df = pd.concat(rows_to_concat).sort_index()

In [37]:
# Aggregate sentiment scores per country per quarter to obtain sentiment index
df_sentiment_index = pd.DataFrame(df[countries_list].groupby(df.index).sum())

In [38]:
# Remove years 2002 and 2023, as they are incomplete
df_sentiment_index = df_sentiment_index.loc[(df_sentiment_index.index.year != 2002) & (df_sentiment_index.index.year != 2023)]

In [6]:
# Create column that contains aggregate sentiment of all countries not weighted by country size 
df_sentiment_index["unweighted_aggregate"] = df_sentiment_index.sum(axis = 1)

In [39]:
filtered_df = df.nsmallest(100, 'greece')

# Select only the desired columns
filtered_df = filtered_df[['company', 'date', 'country_count', "greece", "quarter"]]

print(filtered_df[filtered_df.index.year == 2010])

                 company        date country_count  greece quarter
2010-03-31  CTIP3.SA^C17  2010-05-14             1    -3.0      Q1
2010-12-31          AB.N  2011-02-10             3    -3.0      Q4
2010-03-31   GFNORTEO.MX  2010-04-30             1    -2.0      Q1
2010-03-31         MBI.N  2010-05-11             1    -2.0      Q1
2010-12-31     TCF.N^H19  2011-01-20             1    -2.0      Q4
2010-12-31     FIG.N^L17  2011-03-01             2    -2.0      Q4


In [35]:
erst = df[df["company"] == "ERST.VI"]
erst = erst[['company', 'date', 'country_count', "greece", "quarter"]]
print(erst.sort_values("greece"))

            company        date country_count    greece quarter
2011-06-30  ERST.VI  2011-07-29           120 -0.100000      Q2
2010-03-31  ERST.VI  2010-04-30            55 -0.036364      Q1
2014-12-31  ERST.VI  2015-02-27           160 -0.025000      Q4
2009-03-31  ERST.VI  2009-04-30            77 -0.012987      Q1
2012-09-30  ERST.VI  2012-10-30           111 -0.009009      Q3
...             ...         ...           ...       ...     ...
2009-06-30  ERST.VI  2009-07-30            66  0.000000      Q2
2009-03-31  ERST.VI  2009-04-30            46  0.000000      Q1
2011-09-30  ERST.VI  2011-10-28           121  0.000000      Q3
2022-12-31  ERST.VI  2022-08-01            60  0.000000      HY
2017-06-30  ERST.VI  2017-08-04            98  0.030612      Q2

[105 rows x 5 columns]


### First stage: Validation of sentiment index and construction of unwarranted sentiment

In [ ]:
## Validation: sentiment index must be related to fundamentals, r-square important --> create graph

In [ ]:
## If validated: take residuals as unwarranted sentiment measure

### Second stage: Drivers of bond yield spreads

In [ ]:
## Regress unwarranted sentiment and fundamentals on bond yield spreads, r-square important

### Graphs, Figures, and Tables

In [49]:
## Graph: debt-to-gdp

# Select columns for graph: Germany, Italy, France, Greece, and Spain
selected_columns = ["Germany", "Italy", "France", "Greece", "Spain"]
debt_to_gdp_graph = debt_to_gdp[selected_columns]

# Plot data as line chart
fig, ax = plt.subplots(figsize=(8, 6))

# Set line colors and styles
colors = plt.cm.Reds(np.linspace(0, 1, len(debt_to_gdp_graph.columns)))
line_styles = ['-', '--', '-.', ':', '--']

# Plot each country's data as a separate line
for i, col in enumerate(debt_to_gdp_graph.columns):
    ax.plot(debt_to_gdp_graph.index, debt_to_gdp_graph[col], color = colors[i], linestyle = line_styles[i], linewidth = 2, label = col

# Set y-axis label
ax.set_ylabel("Debt-to-GDP Ratio (\%)", fontsize = 12)

# Set y-axis format as percentage
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: "{:.0%}".format(y/100)))

# Add legend to plot
ax.legend(loc = "upper center", bbox_to_anchor = (0.5, -0.1), ncol = 5)

# Save plot as PDF file
plt.savefig(r"figures\debt_to_gdp_plot.pdf", bbox_inches = "tight")

# Do not show graph
plt.close()

In [50]:
## Graph: yield spreads

# Select columns for graph: Germany, Italy, France, Greece, and Spain
selected_columns = ["italy", "france", "greece", "spain"]
yield_spreads_graph = yield_spreads[selected_columns]

# Plot data as line chart
fig, ax = plt.subplots(figsize = (8, 6))

# Set line colors and styles
colors = plt.cm.Reds(np.linspace(0, 1, len(yield_spreads_graph.columns)))
line_styles = ['-', '--', '-.', ':', '--']

# Plot each country's data as a separate line
for i, col in enumerate(yield_spreads_graph.columns):
    ax.plot(yield_spreads_graph.index, yield_spreads_graph[col], color = colors[i], linestyle = line_styles[i], linewidth = 2, label = col.capitalize())

# Set y-axis label
ax.set_ylabel("Bond Yield Spread (\%)", fontsize = 12)

# Set y-axis format as percentage
ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: "{:.0%}".format(y/100)))

# Add legend to plot
ax.legend(loc = "upper center", bbox_to_anchor=(0.5, -0.1), ncol=5)

# Save plot as PDF file
plt.savefig(r"figures\yield_spreads_plot.pdf", bbox_inches = "tight")

# Do not show graph
plt.close()

In [73]:
## Graphs: sentiment index per country

# Plot sentiment as line chart for each EU country
for country in countries_list:
    
    if country in df_sentiment_index.columns:
        
        # Select columns for graph: Germany, Italy, France, Greece, and Spain
        selected_columns = [country]
        df_index_graph = df_sentiment_index[selected_columns]

        # Plot data as line chart
        fig, ax = plt.subplots(figsize = (8, 6))

        # Set line colors and styles
        colors = plt.cm.Reds(np.linspace(0, 1, len(df_index_graph.columns)) + 1)
        line_styles = ['-', '--', '-.', ':', '--']

        # Plot each country's data as a separate line; invert score such that negative sentiment is associated with positive y-values 
        for i, col in enumerate(df_index_graph.columns):
            ax.plot(df_index_graph.index, -df_index_graph[col], color = colors[i], linestyle = line_styles[i], linewidth = 1.5, label = col.capitalize())

            # Shade negative sentiment red and positive sentiment green
            ax.fill_between(df_index_graph.index, -df_index_graph[col], 0, where = (-df_index_graph[col] >= 0), color = "red", alpha = 0.1, interpolate = True)
            ax.fill_between(df_index_graph.index, -df_index_graph[col], 0, where = (-df_index_graph[col] <= 0), color = "green", alpha = 0.1, interpolate = True)

        # Add horizontal line at y=0
        ax.axhline(0, color = "grey", linewidth = .8, zorder = 1)

        # Set y-axis label
        ax.set_ylabel("{} Sentiment Score (inverted)".format(country.title()), fontsize = 12)
        
        # Remove top and right lines of box
        # ax.spines["top"].set_visible(False)
        # ax.spines["right"].set_visible(False)
        
        # Save plot as PDF file and close plot
        plt.savefig(r"figures\sentiment_index\{}_sentiment_plot.pdf".format(country), bbox_inches = "tight")
        plt.close(fig)

### Notes

In [ ]:
## TO DO

# Also get interest rates + QE from Eikon?
# Get remaining transcripts from eikon
# Delete questions from analysts?
# +/- 10 words algorithm cannot overlap answers?